In [15]:
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# 1. Implementaci´on
Implemente el modelo en pymc3 para los priors Beta(1,1) y Beta(100,100), y grafique las posteriors resultantes de observar 6 caras de 6 tiradas.

In [32]:
def inferencia_moneda(alfa, beta, n=6, obs=6):
    modelo = pm.Model()
    with modelo:
        theta = pm.Beta('theta', alfa, beta)
        coin_obs = pm.Binomial('co', n=n, p=theta, observed=obs)

#         start = pm.find_MAP()
#         print start
        step = pm.Metropolis()
        trace = pm.sample(1000, step)
    
#     print trace['theta']
    plt.hist(trace['theta'], 15, histtype='step', normed=True, label='post');
    x = np.linspace(0, 1, 100)
    plt.plot(x, stats.beta.pdf(x, alfa, beta), label='prior');
    plt.legend(loc='best');
    plt.show()

In [33]:
# inferencia_moneda(1,1)
inferencia_moneda(100,100)

100%|██████████| 1500/1500 [00:00<00:00, 11242.61it/s]


ValueError: max must be larger than min in range parameter.

# 2. Res´umenes
Calcule el MAP (m´aximo a posteriori) y las medias de ambas posteriors.

# 3. Predicci´on
Prediga, para ambos priors, el resultado de una nueva tirada de moneda.

# 4. Otros datos
Repita los puntos anteriores para una situaci´on en la que se observan 20 caras de 20 tiradas, y otra en
la que se observan 3 caras de 3 tiradas. ¿Qu´e puede concluir de los priors elegidos? ¿Y del modelo en
general?

# 5. Prior
Pensando en su propia experiencia, considere cu´antas caras consecutivas le har´ıan empezar a sospechar
que la moneda est´a cargada. A partir de este n´umero y usando el modelo, diga cu´ales son los par´ametros
de su prior Beta.